In [1]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np

# -----------------------------------------------------------------------------
# Paths
# -----------------------------------------------------------------------------
BASE_DIR = Path("..")
DATA_DIR = BASE_DIR / "data"
DB_PATH = DATA_DIR / "housing.db"

# -----------------------------------------------------------------------------
# Load raw CSV
# -----------------------------------------------------------------------------
housing = pd.read_csv(DATA_DIR / "housing" / "housing.csv")

# -----------------------------------------------------------------------------
# Create classification target
# -----------------------------------------------------------------------------
housing["price_class"] = pd.qcut(
    housing["median_house_value"],
    q=3,
    labels=["LOW", "MEDIUM", "HIGH"]
)

# -----------------------------------------------------------------------------
# Add surrogate key
# -----------------------------------------------------------------------------
housing = housing.reset_index().rename(columns={"index": "block_id"})

# -----------------------------------------------------------------------------
# Dimension: ocean_proximity
# -----------------------------------------------------------------------------
ocean_dim = (
    housing[["ocean_proximity"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
ocean_dim["ocean_proximity_id"] = ocean_dim.index + 1

housing = housing.merge(ocean_dim, on="ocean_proximity", how="left")

# -----------------------------------------------------------------------------
# Dimension: price_class
# -----------------------------------------------------------------------------
price_dim = (
    housing[["price_class"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
price_dim["price_class_id"] = price_dim.index + 1

housing = housing.merge(price_dim, on="price_class", how="left")

# -----------------------------------------------------------------------------
# Create 3NF tables
# -----------------------------------------------------------------------------
df_ocean = ocean_dim.rename(columns={"ocean_proximity": "name"})[
    ["ocean_proximity_id", "name"]
]

df_price = price_dim.rename(columns={"price_class": "label"})[
    ["price_class_id", "label"]
]

df_block = housing[
    ["block_id", "longitude", "latitude", "ocean_proximity_id"]
]

df_stats = housing[
    [
        "block_id",
        "housing_median_age",
        "total_rooms",
        "total_bedrooms",
        "population",
        "households",
        "median_income",
        "median_house_value",
        "price_class_id",
    ]
]

# -----------------------------------------------------------------------------
# Build SQLite DB
# -----------------------------------------------------------------------------
if DB_PATH.exists():
    DB_PATH.unlink()

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.executescript("""
DROP TABLE IF EXISTS block_housing_stats;
DROP TABLE IF EXISTS block;
DROP TABLE IF EXISTS ocean_proximity;
DROP TABLE IF EXISTS price_class;

CREATE TABLE ocean_proximity (
    ocean_proximity_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL UNIQUE
);

CREATE TABLE price_class (
    price_class_id INTEGER PRIMARY KEY,
    label TEXT NOT NULL UNIQUE
);

CREATE TABLE block (
    block_id INTEGER PRIMARY KEY,
    longitude REAL NOT NULL,
    latitude REAL NOT NULL,
    ocean_proximity_id INTEGER NOT NULL,
    FOREIGN KEY (ocean_proximity_id)
        REFERENCES ocean_proximity(ocean_proximity_id)
);

CREATE TABLE block_housing_stats (
    block_id INTEGER PRIMARY KEY,
    housing_median_age REAL NOT NULL,
    total_rooms INTEGER NOT NULL,
    total_bedrooms INTEGER,
    population INTEGER NOT NULL,
    households INTEGER NOT NULL,
    median_income REAL NOT NULL,
    median_house_value REAL NOT NULL,
    price_class_id INTEGER NOT NULL,
    FOREIGN KEY (block_id)
        REFERENCES block(block_id),
    FOREIGN KEY (price_class_id)
        REFERENCES price_class(price_class_id)
);
""")

cur.executemany(
    "INSERT INTO ocean_proximity VALUES (?, ?)",
    df_ocean.itertuples(index=False)
)

cur.executemany(
    "INSERT INTO price_class VALUES (?, ?)",
    df_price.itertuples(index=False)
)

cur.executemany(
    "INSERT INTO block VALUES (?, ?, ?, ?)",
    df_block.itertuples(index=False)
)

cur.executemany(
    "INSERT INTO block_housing_stats VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
    df_stats.itertuples(index=False)
)

conn.commit()
conn.close()

print("✅ Classification database built successfully!")


✅ Classification database built successfully!


In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/housing.db")

df = pd.read_sql_query(
    """
    SELECT
        pc.label AS price_class,
        COUNT(*) AS count
    FROM block_housing_stats bhs
    JOIN price_class pc
        ON pc.price_class_id = bhs.price_class_id
    GROUP BY pc.label
    """,
    conn
)

conn.close()
df


,price_class,count
0,HIGH,6880
1,LOW,6884
2,MEDIUM,6876
